# SEC Filings and Contracts Analysis with BigQuery ML/AI

This notebook demonstrates my initial setup to try out BigQuery's AI/ML capabilities to transform unstructured SEC filings and contract documents into structured data using bigquery's AI.GENERATE and AI.GENERATE_TABLE

## DATASET: 

<b>Public data</b> from [Material Contract Corpus - Stanford University](https://mcc.law.stanford.edu/download/contracts/) was used to create a working prototype with the AI/ML capabilities of BigQuery. 
- It is publicly available. 
- Contains over 1M files ranging from the year 2000 to 2023. 
- Due to it's scale and costs, I am using files from the year 2020 to 2023. (130k Documents) to showcase bigquery's AI capabilities. 


I have ingested these documents onto my BUCKET. 

## Goal

My goal is to 

- reduce the time taken to find enterprise documents
- be able to answer questions about data
- To build the solution with bigquery

Every organization has unstrctured data like txt/html/pdf files. I want to use AI to parse these into a semi-strucutred way, so I can be able to query them (there is a reason SQL is popular), instead of having to do RAG all the time. 

The problem with RAG is that I always get something, but it might not be relevant. And the LLM would have to take all the input in and come up with an answer. Instead we can reduce the context and make it more revelant if we go with a hybrid approach. 

By making a schema and parsing our data, and also by making an embedding of the summary generated. We can retrieve necessary documents. 
For speciliazed questions. We can use AI.GENERATE_BOOL and AI.GENERATE to do a sentiment analysis /filter things out. 

This way, we can solve the documents problems with very organization that has huge amounts of data. 

## Prerequisites
1. Google Cloud Project with BigQuery and Vertex AI APIs enabled
2. Contract documents uploaded to GCS bucket (`contracts-mcc`). This is PUBLIC DATA. [Material Contract Corpus - Stanford University](https://mcc.law.stanford.edu/download/contracts/)
3. Proper authentication (gcloud auth login or service account)


In [1]:
# Install required packages (run this once)
# !uv pip install google-cloud-bigquery pandas db-dtypes

In [1]:
import os
import json
import pandas as pd
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import time

In [2]:
# Configuration
PROJECT_ID = "cool-automata-386721"  # Your Google Cloud Project ID
GCS_BUCKET_NAME = "contracts-mcc"  # Your GCS bucket with contract documents
DATASET_NAME = "exploration"

# Initialize BigQuery client
client = bigquery.Client(project=PROJECT_ID)


### Create BigQuery Cloud resource connection and service account permissions

In [4]:
!bq mk --connection --location=us \
    --connection_type=CLOUD_RESOURCE contracts_ai_connection

BigQuery error in mk operation: Already Exists: Connection
projects/625945884811/locations/us/connections/contracts_ai_connection


In [25]:
SERVICE_ACCT = !bq show --format=prettyjson --connection us.contracts_ai_connection | grep "serviceAccountId" | cut -d '"' -f 4
SERVICE_ACCT_EMAIL = SERVICE_ACCT[-1]
print(SERVICE_ACCT_EMAIL)

bqcx-625945884811-s8gk@gcp-sa-bigquery-condel.iam.gserviceaccount.com


In [6]:
import time

!gcloud projects add-iam-policy-binding --format=none $PROJECT_ID --member=serviceAccount:$SERVICE_ACCT_EMAIL --role='roles/storage.objectViewer'
!gcloud projects add-iam-policy-binding --format=none $PROJECT_ID --member=serviceAccount:$SERVICE_ACCT_EMAIL --role='roles/aiplatform.user'

# Wait ~60 seconds, to give IAM updates time to propagate. Otherwise, subsequent cells may fail.
time.sleep(60)

Updated IAM policy for project [cool-automata-386721].
Updated IAM policy for project [cool-automata-386721].


## Step 1: Create Dataset and the table


We will be creating an exploration dataset, where the raw data will be an object table sec_filings,
and the parsed data will go into contracts table with the schema below.  - Initial Idea



In [20]:
dataset_id = f"{PROJECT_ID}.{DATASET_NAME}"

try:
    dataset = client.get_dataset(dataset_id)
except NotFound:
    dataset = bigquery.Dataset(dataset_id)
    dataset.location = "US"
    dataset = client.create_dataset(dataset, timeout=30)

# Create the contracts table with your desired schema
table_id = f"{dataset_id}.contracts"

schema = [
    # Filing-Level Context (Denormalized onto each contract)
    # This data is repeated for each contract from the same filing, denormalized for bigQuery.

    bigquery.SchemaField("contract_id", "STRING"),        # Your unique ID for this specific contract 
    bigquery.SchemaField("company_name", "STRING"),       # Company name
    bigquery.SchemaField("form_type", "STRING"),          # e.g., '10-K', '8-K'
    bigquery.SchemaField("filing_date", "DATE"),          # Filing date
    bigquery.SchemaField("state_incorp", "STRING"),       # Company's state of incorporation

    bigquery.SchemaField("contract_type", "STRING"),      # 'Employment', 'Lease', 'Purchase/M&A', etc.
    bigquery.SchemaField("contract_date", "DATE"),        # The execution date of the contract
    bigquery.SchemaField("governing_law_state", "STRING"),# e.g., 'DE', 'CA', crucial for legal queries
    bigquery.SchemaField("contract_summary", "STRING"),   # AI-generated summary of the contract's purpose and key details, like parties, purpose etc.
    bigquery.SchemaField("numeric_value", "NUMERIC"),  # Value of the contract/agreement

    bigquery.SchemaField("parties", "RECORD", mode="REPEATED", fields=[
        bigquery.SchemaField("name", "STRING"),
    ]),
    bigquery.SchemaField("clauses", "RECORD", mode="REPEATED", fields=[
        bigquery.SchemaField("clause_type", "STRING"),    # e.g., 'Change of Control', 'Auto-Renewal'
    ]),

    bigquery.SchemaField("file_path", "STRING"), #GCS file path
]

try:
    table = client.get_table(table_id)
    print(f"✅ Table {table_id} already exists")
except NotFound:
    table = bigquery.Table(table_id, schema=schema)
    table = client.create_table(table)
    print(f"✅ Created table {table_id}")


✅ Table cool-automata-386721.exploration.contracts already exists


## Step 2: Create Object Table for GCS Objects

This creates an "object table" that gives us ObjectRef access to files in GCS.


In [3]:
%load_ext google.cloud.bigquery

/Users/Meher.Gajula/Desktop/Work/playground/BQ Comp 2025/.venv/lib/python3.13/site-packages/google/cloud/bigquery/__init__.py:237: FutureWarning: %load_ext google.cloud.bigquery is deprecated. Install bigquery-magics package and use `%load_ext bigquery_magics`, instead.
  warnings.warn(


In [4]:
# !uv pip install tqdm

I am trying to make an object table which just the 2020 Q1 Documents, this is about 11k documents to explore AI.GENERATE_TEXT AND AI.GENERATE_TABLE

In [7]:
object_table_id = f"exploration.sec_filings"
from google.cloud import storage

storage_client = storage.Client()
bucket = storage_client.bucket(GCS_BUCKET_NAME)

# List files in the production_contracts directory
blobs = list(bucket.list_blobs(prefix="2020/Q1/"))
print(f"Found {len(blobs)} files in 2020/Q1/")

if blobs:
    print("Sample files:")
    for blob in blobs[:5]:
        print(f"  📄 {blob.name} ({blob.size:,} bytes)")

Found 11311 files in 2020/Q1/
Sample files:
  📄 2020/Q1/1000209..000119312520022930..d879007dex101.htm (9,778 bytes)
  📄 2020/Q1/1000209..000156459020014310..mfin-ex1042_497.htm (15,820 bytes)
  📄 2020/Q1/1000228..000100022820000018..d848607dex1018.htm (4,884 bytes)
  📄 2020/Q1/1000228..000100022820000018..d848607dex1032.htm (4,695 bytes)
  📄 2020/Q1/1000229..000156459020008962..clb-ex101_6.htm (9,936 bytes)


### Create the Object table

In [5]:
%%bigquery --project {PROJECT_ID}
CREATE OR REPLACE EXTERNAL TABLE `exploration.sec_filings`
WITH CONNECTION `us.contracts_ai_connection`
OPTIONS (
  object_metadata = 'SIMPLE',
  uris = ['gs://contracts-mcc/2020/Q1/*']
)

Query is running:   0%|          |

""


In [8]:
# Let's check what files we have access to
query = f"""
SELECT count (*)
FROM `{object_table_id}`
"""

try:
    print("Checking available sec filings...")
    count = client.query(query).to_dataframe()
    print(f"✅ Found {count.iloc[0]} documents")
except Exception as e:
    print(f"Error querying documents: {e}")


Checking available sec filings...


E0000 00:00:1758475320.710022 25380281 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


✅ Found f0_    11311
Name: 0, dtype: Int64 documents


## Step 3: AI.GENERATE and AI.GENERATE_TABLE

Now we'll use BigQuery's AI functions to extract structured information from the contract documents. We'll start with a small sample.


### Create a Gemini Model for Bigquery

In [10]:
# Create a remote model for Gemini (required for AI functions)
model_id = f"exploration.gemini"

create_model_sql = f"""
CREATE OR REPLACE MODEL `{model_id}`
REMOTE WITH CONNECTION `us.contracts_ai_connection`
OPTIONS (ENDPOINT = 'gemini-2.0-flash')
"""

try:
    query_job = client.query(create_model_sql)
    query_job.result()
except Exception as e:
    print(f"Error creating model: {e}")


### Parsing with AI.GENERATE 

In [11]:
# FIXED: Enhanced AI parsing with correct AI.GENERATE syntax
enhanced_test_query = f"""
SELECT
  uri,
  AI.GENERATE(
    (
      'You are an expert SEC filing analyst. Analyze this document and EXTRACT the following information in VALID JSON format only:' ||
      '{{"company_name": "Full legal company name or null if not found",' ||
      '"document_type": "SEC form type (10-K, 10-Q, 8-K, S-1, etc.) or contract type",' ||
      '"filing_info": {{"cik": "CIK number or null", "form_type": "form type", "filing_date": "YYYY-MM-DD format or null"}},' ||
      '"contract_details": {{"contract_type": "type of agreement/contract or null", "execution_date": "YYYY-MM-DD format or null"}},' ||
      '"parties": ["array of party names involved or empty array"],' ||
      '"key_clauses": ["array of important clause types mentioned or empty array"]}}' ||
      'IMPORTANT: Return ONLY valid JSON, no markdown, no explanations.',
      ref
    ),
    connection_id => 'us.contracts_ai_connection',
    endpoint => 'gemini-2.0-flash'
  ).result as parsed_json
FROM `{object_table_id}`
LIMIT 3
"""


df = client.query(enhanced_test_query).to_dataframe()
df.head()

E0000 00:00:1758475351.673064 25380281 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


,uri,parsed_json
0,gs://contracts-mcc/2020/Q1/1000228..0001000228...,"```json\n{""company_name"": ""HENRY SCHEIN, INC.""..."
1,gs://contracts-mcc/2020/Q1/1000209..0001193125...,"```json\n{""company_name"": ""FRESHSTART VENTURE ..."
2,gs://contracts-mcc/2020/Q1/1000209..0001564590...,"```json\n{""company_name"": ""FRESHSTART VENTURE ..."


#### Better Prompting with Langchain and Pydantic schema


I want to extract details, and to better add context, I think adding descriptions and types through pydantic can be better.

In [ ]:
# !uv pip install langchain pydantic

In [12]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from typing import List, Optional

# Single denormalized Pydantic model matching your BigQuery schema
class Contract(BaseModel):
    company_name: Optional[str] = Field(description="Company name - most queried field")
    form_type: Optional[str] = Field(description="SEC form type like '10-K', '8-K'")
    filing_date: Optional[str] = Field(description="Filing date in YYYY-MM-DD format")
    state_incorp: Optional[str] = Field(description="Company's state of incorporation")
    
    # Contract-Level Fields
    contract_type: Optional[str] = Field(description="Contract type: 'Employment', 'Lease', 'Purchase/M&A', etc.")
    contract_date: Optional[str] = Field(description="Contract execution date in YYYY-MM-DD format")
    governing_law_state: Optional[str] = Field(description="Governing law state like 'DE', 'CA'")
    contract_summary: Optional[str] = Field(description="AI-generated summary of contract's purpose")
    numeric_value: Optional[float] = Field(description="Value of the contract/agreement")
    
    # Flattened Arrays (BigQuery will handle as REPEATED fields)
    parties: List[str] = Field(description="List of party names involved", default=[])
    clauses: List[str] = Field(description="List of clause types like 'Change of Control', 'Auto-Renewal'", default=[])

# Setup the parser and prompt template
json_parser = JsonOutputParser(pydantic_object=Contract)
format_instructions = json_parser.get_format_instructions()

extraction_prompt_template = PromptTemplate(
    input_variables=["doc_type", "format_instructions"],
    template="""You are an expert SEC filing and contract analyst. Extract information from this document.

Document Type: {doc_type}

{format_instructions}

CRITICAL INSTRUCTIONS:
- Extract ALL available information, even if some fields are null
- For dates, use YYYY-MM-DD format only
- For numeric_value, extract any dollar amounts mentioned
- For parties, include all company/individual names involved
- For clauses, identify key clause types like termination, renewal, change of control, etc.
- If this is an SEC filing containing multiple contracts, focus on the most significant one
- Use null for fields not found or not applicable

Return ONLY valid JSON, no markdown, no explanations."""
)

extraction_prompt = extraction_prompt_template.format(doc_type="contract", format_instructions=format_instructions)
print(extraction_prompt)


You are an expert SEC filing and contract analyst. Extract information from this document.

Document Type: contract

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"company_name": {"anyOf": [{"type": "string"}, {"type": "null"}], "description": "Company name - most queried field", "title": "Company Name"}, "form_type": {"anyOf": [{"type": "string"}, {"type": "null"}], "description": "SEC form type like '10-K', '8-K'", "title": "Form Type"}, "filing_date": {"anyOf": [{"type": "string"}, {"type": "null"}], "description": "Filing date in YYYY-MM-DD format", "title": "Filing Da

Although, I have a better parsing prompt, I had trouble adding it in the biquery sql scripts and functions, and maybe I should tryout through python in the future. 

### Using AI.GENERATE_TABLE to parse the documents

In [13]:
%%bigquery --project {PROJECT_ID}

SELECT
uri,
company_name,
form_type,
filing_date,
state_incorp,
contract_type,
contract_date,
governing_law_state,
parties,
clauses,
contract_summary

FROM AI.GENERATE_TABLE(
  MODEL `exploration.gemini`,
  (
    SELECT (
      'You are an expert SEC filing and contract analyst. Analyze this document and EXTRACT the following information in VALID JSON format only:' ||
      '{{"company_name": "Full legal company name or null if not found",' ||
      '"document_type": "SEC form type (10-K, 10-Q, 8-K, S-1, etc.) or contract type",' ||
      '"filing_info": {{"cik": "CIK number or null", "form_type": "form type", "filing_date": "YYYY-MM-DD format or null"}},' ||
      ' "state_incorp": "State of incorporation or null",' ||
      '"contract_details": {{"contract_type": "type of agreement/contract or null", "execution_date": "YYYY-MM-DD format or null"}},' ||
      '"governing_law_state": "State of governing law or null",' ||
      '"parties": ["array of party names involved or empty array"],' ||
      '"clauses": ["array of important clause types mentioned or empty array"]}}' ||
      ' "contract_summary": "A summary of what this contract means, include key parties, and the type of agreement",' ||
      'IMPORTANT: Return ONLY valid JSON, no markdown, no explanations.',
      ref
    ) AS prompt,
    uri,
    FROM `exploration.sec_filings`
  ),
  STRUCT(
     "company_name STRING, form_type STRING, filing_date STRING, state_incorp STRING, contract_type STRING, contract_date STRING, governing_law_state STRING, parties ARRAY<STRING>, clauses ARRAY<STRING>, contract_summary STRING" AS output_schema
  )
)
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,uri,company_name,form_type,filing_date,state_incorp,contract_type,contract_date,governing_law_state,parties,clauses,contract_summary
0,gs://contracts-mcc/2020/Q1/1000229..0001564590...,Core Laboratories N.V.,EX-10.1,2020-03-01,null,Amendment to Employment Agreement,None,null,"[Core Laboratories N.V., Lawrence V. Bruno]",[],None
1,gs://contracts-mcc/2020/Q1/1000228..0001000228...,"HENRY SCHEIN, INC.",EX-10.18,2018-12-20,"null"",\n ""clauses"": [],\n ""contract_summary""...",Amendment,None,New York,"[Henry Schein, Inc., Compensation Committee]",[],None
2,gs://contracts-mcc/2020/Q1/1000209..0001193125...,FRESHSTART VENTURE CAPITAL CORPORATION,EX-10.1,2020-01-30,"NY, New York (principal office location, not n...",Amendment to Loan Agreement,None,New York,"[U.S. SMALL BUSINESS ADMINISTRATION, FRESHSTAR...",[],None
3,gs://contracts-mcc/2020/Q1/1000228..0001000228...,"HENRY SCHEIN, INC.",EX-10.32,2018-12-20,"null"",\n ""governing_law_state"": ""null"",\n ""p...",Amendment to Deferred Compensation Plan,None,None,[],[],None
4,gs://contracts-mcc/2020/Q1/1000209..0001564590...,FRESHSTART VENTURE CAPITAL CORPORATION,EX-10.42,2020-03-27,"NY, principal office located at 437 Madison Av...",Amendment to Loan Agreement,None,New York,"[U.S. SMALL BUSINESS ADMINISTRATION, FRESHSTAR...",[],None


## Feedback: 

- Having a stop job feature in bigquery UI could help, I tried to run the GENERATE_TABLE on a massive table, and I am not sure of the ongoing progress for that job that's running and I wanted to have some control. So I stopped it using a python function. 

- Autocomplete features in bigquery editor would have helped me save time, as I was just missing a ' , ' in output schema. 

- Seeing the token processed info in the bigquery editor would be great, I read the documentation where output_schema returns the full_response, but having it in BQ UI will be great. 


In [14]:
from google.cloud import bigquery


def cancel_job(
    client: bigquery.Client,
    job_id: str = "bquxjob_3824d4ce_1996d68cbd6",
) -> None:
    job = client.cancel_job(job_id)
    print(f"{job.job_id} cancelled")

cancel_job(client)

bquxjob_3824d4ce_1996d68cbd6 cancelled


## Cleanup commands

These commands will clean up the resources created in this notebook


In [ ]:
# !bq rm --table -f exploration.contracts
# !bq rm --table -f exploration.sec_filings

In [ ]:
# !bq rm --model -f exploration.gemini

In [ ]:
# !bq rm --connection --project_id=$PROJECT_ID --location=us contracts_ai_connection